# Entropy

A measure of Impurity. It is given by the formula:
![entropy](entropy.png)

In [1]:
# let's create a function to calculate the entropy of a column in a dataset

from collections import Counter
import math

def shannonentropy(column):
    totalCount = len(column)
    labelCount = dict(Counter(column)) # dictionary of labels with their counts
    entropy = 0
    for label in labelCount:
        pLabel = labelCount[label]/totalCount
        entropy -= pLabel*math.log(pLabel, 2)
    return entropy
    

In [2]:
# let's create our dataset

import pandas as pd

selfCar = pd.DataFrame({"Grade": ['steep', 'steep', 'flat', 'steep'],
                       "Bumpiness": ["bumpy", 'smooth', 'bumpy', 'smooth'],
                       "Speed Limit":['yes', 'yes', 'no', 'no'],
                       "Speed": ['slow', 'slow', 'fast', 'fast']})
selfCar

,Grade,Bumpiness,Speed Limit,Speed
0,steep,bumpy,yes,slow
1,steep,smooth,yes,slow
2,flat,bumpy,no,fast
3,steep,smooth,no,fast


In [3]:
shannonentropy(selfCar['Speed'])

1.0

# Information Gain

Information Gain is a metric which decides on which feature should a Node (The Labels) be split upon

$Information\ Gain$ = $Entropy\ (Parent)$ - $[Weighted\ Average]\ Entropy\ (Children)$

The decision tree algorithm maximizes the information gain.

Think of it as a two dimensional
plot of some data. You want to draw a line to separate one class from
another. Should you do this on the X-axis or the Y-axis? The answer is what you’re trying
to find out here.


In [4]:
# this function returns the information gain if the Class List is split upon by a particular feature
def informationGain(dataset, node, featuretoSplitUpon):
    # weighted entropy is the sum of entropies of child nodes multiplied by their respective probabilities
    weightedEntropy = 0 
    for label in dataset[featuretoSplitUpon].unique():
        child = dataset[dataset[featuretoSplitUpon]==label][node] # the resulting child node if the parent node is
                                                                  # split upon by the given feature
        entropyChild = shannonentropy(child)
        probChild = len(child)/len(dataset[node])
        weightedEntropy += entropyChild * probChild
    entropyParent = shannonentropy(dataset[node])
    return entropyParent - weightedEntropy

In [5]:
informationGain(selfCar, 'Speed', 'Speed Limit')

1.0

In [6]:
# we're assuming that the classList column is at the end of the dataset
def bestFeaturetoSplitUpon(dataset):
    featureIG = {} # this is a dictionary which maps the Information Gain with it's features
    for feature in dataset.columns[:-1]:
        featureIG[feature] = informationGain(dataset, dataset.columns[-1], feature)
    sortedFeatureIG = sorted(featureIG, key=lambda x: x[1]) # sorting the dictionary
    return sortedFeatureIG[0] # return the first element of the sorted dictionary

In [7]:
bestFeaturetoSplitUpon(selfCar)

'Speed Limit'

In [8]:
# let's make another dataset
fish = pd.DataFrame({"Can Survive without Coming to Surface?": ['yes', 'yes', 'yes', 'no', 'no'],
                    "Has Flippers?": ['yes', 'yes', 'no', 'yes', 'yes'],
                    "Fish?": ['yes', 'yes', 'no', 'no', 'no']})
fish

,Can Survive without Coming to Surface?,Has Flippers?,Fish?
0,yes,yes,yes
1,yes,yes,yes
2,yes,no,no
3,no,yes,no
4,no,yes,no


In [9]:
bestFeaturetoSplitUpon(fish)

'Can Survive without Coming to Surface?'

# Decision Tree

In [52]:
# this function builds a decision tree
# it does so by recursively splitting the class labels upon the best feature until the base case is reached

def createDecisionTree(dataset):
    classList = dataset.iloc[:, -1] # the labels class
    # the base case is reached when the class labels are all the same and we're going to return that label
    if classList.nunique() == 1:
        return classList.unique()[0] # this is going to be the leaf node at the bottom
    # when there are no more features to split upon, you return the class label with highest frequency
    if dataset.shape[1] == 1:
        return classList.value_counts(sort=True, ascending=False).index[0]
    bestFeature = bestFeaturetoSplitUpon(dataset) # the best feature to split upon
    myTree = {bestFeature:{}} # initializing the tree # Note: the tree is initialised everytime it's recursed
    # Now we split upon the labels of the best feature
    # and recurse the function on the dataset that resulted from the split 
    for label in dataset[bestFeature].unique():
        # the resulting dataset after the split is made upon a particular label of the best feature
        splitDataset = dataset[dataset[bestFeature]==label].drop(bestFeature, axis=1)
        # run the function recursively on the split dataset
        myTree[bestFeature][label] = createDecisionTree(splitDataset)
        # print(myTree) # use print function to visualise the recursion. This is similar to depth first search
    return myTree

In [53]:
createDecisionTree(fish)

{'Can Survive without Coming to Surface?': {'yes': {'Has Flippers?': {'yes': 'yes',
    'no': 'no'}},
  'no': 'no'}}

In [16]:
selfCar[selfCar['Speed Limit'] == 'yes'].drop('Speed Limit', axis=1)

,Grade,Bumpiness,Speed
0,steep,bumpy,slow
1,steep,smooth,slow


In [17]:
x = {}
x['ya'] = 10
x

{'ya': 10}

In [27]:
fish['Can Survive without Coming to Surface?'].unique()[0]

'yes'

In [30]:
createDecisionTree(selfCar)

{'Speed Limit': {'yes': 'slow', 'no': 'fast'}}

In [31]:
selfCar

,Grade,Bumpiness,Speed Limit,Speed
0,steep,bumpy,yes,slow
1,steep,smooth,yes,slow
2,flat,bumpy,no,fast
3,steep,smooth,no,fast


In [39]:
q = {'a':{}}
q

{'a': {}}

In [40]:
q = {'b':{}}

In [41]:
q

{'b': {}}